# ETL con Datos de Ecobici Buenos Aires

En este notebook se realiza un proceso completo de ETL (Extract, Transform, Load) usando los datos abiertos de Ecobici en la Ciudad de Buenos Aires.

Extracción: descarga y lectura del dataset.

Transformación: limpieza, manejo de nulos, codificación, discretización, normalización y creación de nuevas variables.

Carga: exportar los datos procesados a un archivo nuevo.

In [ ]:
import pandas as pd
import numpy as np
import requests, zipfile, io
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic

print("Librerías listas.")

Librerías listas.


Extracción  de datos

In [ ]:
print("\n--- EXTRACCIÓN ---")

url = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/bicicletas-publicas/recorridos-realizados-2024.zip"
csv_file_name = "badata_ecobici_recorridos_realizados_2024.csv"

response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extract(csv_file_name)

df_raw = pd.read_csv(csv_file_name)
print(f"Datos cargados: {df_raw.shape[0]} registros y {df_raw.shape[1]} columnas.")

df_raw.head()


--- EXTRACCIÓN ---


Transformación de Datos

Limpieza

In [ ]:
df = df_raw.copy()

df['fecha_origen_recorrido'] = pd.to_datetime(df['fecha_origen_recorrido'])
df['fecha_destino_recorrido'] = pd.to_datetime(df['fecha_destino_recorrido'])

print("\nNulos por columna:")
print(df.isnull().sum())

Manejo de nulos

In [ ]:
df['genero'] = df['genero'].fillna("OTHER")

# Eliminar filas con fecha destino nula
df.dropna(subset=['fecha_destino_recorrido'], inplace=True)

print("Nulos resueltos.")

Análisis de variables categóricas

In [ ]:
print("\nConteo por género:")
print(df['genero'].value_counts())

sns.countplot(data=df, x='genero', palette='viridis')
plt.title("Distribución de Género")
plt.show()

Codificación (Encoding)

In [ ]:
df_encoded = pd.get_dummies(df, columns=['genero','modelo_bicicleta'], prefix=['gen','modelo'])
print("Variables categóricas codificadas.")
df_encoded.head()

Discretización

In [ ]:
bins = [0,1200,2400,3600, df['duracion_recorrido'].max()]
labels = ["Corto","Medio","Largo","Muy Largo"]
df_encoded['categoria_duracion'] = pd.cut(df['duracion_recorrido'], bins=bins, labels=labels)

print("Discretización de duración lista.")
df_encoded['categoria_duracion'].value_counts()

Normalización

In [ ]:
scaler = MinMaxScaler()
cols_norm = ["duracion_recorrido","lat_estacion_origen","long_estacion_origen",
             "lat_estacion_destino","long_estacion_destino"]

df_encoded[cols_norm] = scaler.fit_transform(df_encoded[cols_norm])
print("Columnas normalizadas.")

Feature Engineering

In [ ]:
# Duración en minutos
df_encoded['duracion_min'] = (df['fecha_destino_recorrido'] - df['fecha_origen_recorrido']).dt.total_seconds()/60

# Día de la semana y hora de inicio
df_encoded['dia_semana'] = df['fecha_origen_recorrido'].dt.dayofweek
df_encoded['hora_inicio'] = df['fecha_origen_recorrido'].dt.hour

# Tipo de día
df_encoded['tipo_dia'] = df_encoded['dia_semana'].apply(lambda x: "Fin de Semana" if x>=5 else "Entre Semana")

# Distancia geodésica (usamos coordenadas originales)
df_encoded['distancia_km'] = df.apply(lambda row: geodesic(
    (row['lat_estacion_origen'],row['long_estacion_origen']),
    (row['lat_estacion_destino'],row['long_estacion_destino'])
).km, axis=1)

print("Nuevas variables creadas.")
df_encoded.head()

Carga de Datos

In [ ]:
print("\n--- CARGA ---")

df_encoded.to_csv("viajes_ecobici_limpios.csv", index=False)
df_encoded.to_parquet("viajes_ecobici_limpios.parquet", index=False)

print("Datos guardados en CSV y Parquet.")